In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from timeit import default_timer
import time
import json

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split = ['train', 'test'],
    shuffle_files = True,
    as_supervised = True,
    with_info = True,
)

2022-11-27 20:01:54.230979: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., tf.one_hot(label, depth = 10)

ds_train = ds_train.map(normalize_img, num_parallel_calls = tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [4]:
ds_test = ds_test.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [5]:
# Creating the Model...
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape = (28, 28)),
  tf.keras.layers.Dense(128, activation = 'relu'),
  tf.keras.layers.Dense(10)
])

In [6]:
# Compiling the Model
model.compile(
    optimizer = tf.keras.optimizers.Adam(1e-3),
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True),
    metrics = [tf.keras.metrics.CategoricalAccuracy()],
)

In [11]:
class TimingCallback(tf.keras.callbacks.Callback):
    def __init__(self, logs = {}):
        self.logs = []
        
        self.start_list = []
        self.end_list = []
        self.difference_list = []
        
        self.start_train_time = 0.0
        self.end_train_time = 0.0
        
        self.start_batch_interference_list = []
        self.end_batch_interference_list = []
        self.difference_batch_interference_list = []
        
    def on_train_begin(self, logs = {}):
        self.start_train_time = time.time()
    def on_train_end(self, logs = {}):
        self.end_train_time = time.time()
        
    def on_train_batch_begin(self, batch, logs = {}):
        self.starttime = time.time()
        self.start_batch_interference_list.append(self.starttime)
    def on_train_batch_end(self, batch, logs = {}):
        self.endtime = time.time()
        self.end_batch_interference_list.append(self.endtime)
        self.difference_batch_interference_list.append(self.endtime - self.starttime)
        
    def on_epoch_begin(self, epoch, logs = {}):
        self.starttime = time.time()
        self.start_list.append(self.starttime)
    def on_epoch_end(self, epoch, logs = {}):
        self.endtime = time.time()
        self.end_list.append(self.endtime)
        self.difference_list.append(self.endtime - self.starttime)
            
cb = TimingCallback()

In [13]:
# Fitting the Model...
epoch_count = 10
batch_count = 469

model.fit(
    ds_train,
    epochs = epoch_count,
    validation_data = ds_test,
    callbacks = [cb]
)

def average_calculator(ls, num):
    total = 0.0
    average = 0.0
    for val in range(len(ls)):
        total += ls[val]
    average = total / num
    return average

total_training_time = cb.end_train_time - cb.start_train_time
average_epoch_training_time = average_calculator(cb.difference_list, epoch_count)
average_batch_interference_time = average_calculator(cb.difference_batch_interference_list, batch_count)

print("\n", metrics)

metrics = {
    "model_name": "MLP",
    "framework_name": "TensorFlow",
    "dataset": "MNIST Digits",
    "task": "classification",
    "Total Training Time" : total_training_time,
    "Average Epoch Training Time" : average_epoch_training_time,
    "Average Batch Inference Time" : average_batch_interference_time
}

with open("output.json", "w") as outfile:
    json.dump(metrics, outfile)

Epoch 1/10
469/469 [==============================] - 1s 1ms/step - loss: 0.0093 - categorical_accuracy: 0.9969 - val_loss: 0.0931 - val_categorical_accuracy: 0.9801
Epoch 2/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0015 - categorical_accuracy: 0.9999 - val_loss: 0.0838 - val_categorical_accuracy: 0.9824
Epoch 3/10
469/469 [==============================] - 1s 2ms/step - loss: 6.0174e-04 - categorical_accuracy: 1.0000 - val_loss: 0.0863 - val_categorical_accuracy: 0.9820
Epoch 4/10
469/469 [==============================] - 1s 2ms/step - loss: 5.7531e-04 - categorical_accuracy: 1.0000 - val_loss: 0.0857 - val_categorical_accuracy: 0.9824
Epoch 5/10
469/469 [==============================] - 1s 1ms/step - loss: 4.6926e-04 - categorical_accuracy: 1.0000 - val_loss: 0.0850 - val_categorical_accuracy: 0.9820
Epoch 6/10
469/469 [==============================] - 1s 1ms/step - loss: 3.4670e-04 - categorical_accuracy: 1.0000 - val_loss: 0.0855 - val_categorical_accur

In [9]:
model.evaluate(
    ds_train
)

469/469 [==============================] - 0s 818us/step - loss: 0.0227 - categorical_accuracy: 0.9945


[0.022661486640572548, 0.9945499897003174]